In [111]:
# Dependencies
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import itertools
metadata = MetaData()
from config import username, password

In [112]:
dfPop2000 = pd.DataFrame
csvFile = 'Resources/population_by_zip_2000.csv'
dfPop2000 = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfPop2010 = pd.DataFrame
csvFile = 'Resources/population_by_zip_2010.csv'
dfPop2010 = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfBars = pd.DataFrame
csvFile = 'Resources/8260_1.csv'

dfBars = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfBars.dropna(subset=['postalCode'],inplace=True) # Drop rows with no zip code listed
dfBars['postalCode'] = dfBars['postalCode'].str.zfill(5) # Prepend '0' if zip code less than 5 digits long
dfBars['postalCode'] = dfBars['postalCode'].str.slice(start=0,stop=5) # Keep only first 5 digits of zip code
bars_columns = [u'postalCode',u'name']
bars_clean = dfBars[bars_columns].copy()
dfBars=bars_clean.rename(columns={'postalCode':'zipcode'})



ZipBars = dfBars.groupby(['zipcode']).count()
ZipBars.sort_values('name',ascending=False,inplace=True)
ZipBars.rename(columns={'name':'bar_count'},inplace=True)

dfPop2010.dropna(subset=['zipcode'],inplace=True)
dfPop2010['zipcode'] = dfPop2010['zipcode'].str.zfill(5)
dfPop2010['zipcode'] = dfPop2010['zipcode'].str.slice(start=0,stop=5)

dfPop2000.dropna(subset=['zipcode'],inplace=True)
dfPop2000['zipcode'] = dfPop2000['zipcode'].str.zfill(5)
dfPop2000['zipcode'] = dfPop2000['zipcode'].str.slice(start=0,stop=5)


In [113]:
# Create connection to Bars_db
rds_connection_string = username + ':' + password + '@127.0.0.1/Bars_db'
engine = create_engine(f'mysql://{rds_connection_string}',echo=False)
conn = engine.connect()
Base = declarative_base()

In [114]:
metadata = MetaData()
bar_names = Table('bar_names', metadata,
    Column('id', Integer, primary_key=True,nullable=False),
                  Column('zipcode', String(5)),
                  Column('name', String(50))
                 )
metadata.create_all(engine)

In [115]:
engine.table_names()

['bar_names', 'bars_per_zipcode']

In [121]:
listToWrite.iteritems()



TypeError: 'list' object is not callable

In [105]:
#Declaration of the class in order to write into the database. This structure is standard and should align with SQLAlchemy's doc.
class Current(Base):
    __tablename__ = 'bar_names'

    id = Column(Integer, primary_key=True)
    zipcode = Column(String(5))
    name = Column(String(50))

    def __repr__(self):
        return "(id='%s', zipcode='%s', name='%s')" % (self.id, self.zipcode, self.name)

tableToWriteTo = 'bar_names'
listToWrite = dfBars.to_dict(orient='records')
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)

# Open the session
Session = sessionmaker(bind=engine)
session = Session()

# Inser the dataframe into the database in one bulk
conn.execute(table.insert(), listToWrite)

# Commit the changes
session.commit()

# Close the session
session.close()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2019' in position 4: ordinal not in range(256)

In [70]:
ZipBars.to_sql(name='bars_per_zipcode', con=engine, if_exists='replace', index=True, index_label='zipcode',dtype={'zipcode':String(5)})
pd.read_sql_query('select * from bars_per_zipcode', con=engine).head()

,zipcode,bar_count
0,04412,57
1,93446,51
2,94558,49
3,95448,44
4,43215,42


In [71]:
id = dfBars.index.tolist()

In [72]:
dfBars.to_sql(name='bar_names', con=engine, if_exists='replace',index=False,dtype={'id':Integer,'zipcode':String(5),'name':String(50)})
engine.table_names()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2019' in position 4: ordinal not in range(256)

In [73]:
dfBars.head()

,zipcode,name
1,53212,The Waterfront Cafe
2,93612,559 Local Brewing
3,53515,Shack The
4,47130,Flat12 Bierworks
5,98072,Redhook Ale Brewery Inc


In [ ]:
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.orm import sessionmaker
import pandas as pd


# Set up of the engine to connect to the database
# the urlquote is used for passing the password which might contain special characters such as "/"
engine = create_engine('mysql://root:%s@localhost/db1' % urlquote('weirdPassword*withsp€cialcharacters'), echo=False)
conn = engine.connect()
Base = declarative_base()

#Declaration of the class in order to write into the database. This structure is standard and should align with SQLAlchemy's doc.
class Current(Base):
    __tablename__ = 'tableName'

    id = Column(Integer, primary_key=True)
    Date = Column(String(500))
    Type = Column(String(500))
    Value = Column(Numeric())

    def __repr__(self):
        return "(id='%s', Date='%s', Type='%s', Value='%s')" % (self.id, self.Date, self.Type, self.Value)

# Set up of the table in db and the file to import
fileToRead = 'file.csv'
tableToWriteTo = 'tableName'

# Panda to create a lovely dataframe
df_to_be_written = pd.read_csv(fileToRead)
# The orient='records' is the key of this, it allows to align with the format mentioned in the doc to insert in bulks.
listToWrite = df_to_be_written.to_dict(orient='records')

metadata = sqlalchemy.schema.MetaData(bind=engine,reflect=True)
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)

# Open the session
Session = sessionmaker(bind=engine)
session = Session()

# Inser the dataframe into the database in one bulk
conn.execute(table.insert(), listToWrite)

# Commit the changes
session.commit()

# Close the session
session.close()